In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Download TorchVision repo to use some files from
# references/detection
!pip install pycocotools --quiet
!git clone https://github.com/pytorch/vision.git
!git checkout v0.3.0

!cp vision/references/detection/utils.py ./
!cp vision/references/detection/transforms.py ./
!cp vision/references/detection/coco_eval.py ./
!cp vision/references/detection/engine.py ./
!cp vision/references/detection/coco_utils.py ./

Cloning into 'vision'...
remote: Enumerating objects: 531472, done.
remote: Counting objects: 100% (45082/45082), done.
remote: Compressing objects: 100% (2074/2074), done.
remote: Total 531472 (delta 43039), reused 44915 (delta 42935), pack-reused 486390
Receiving objects: 100% (531472/531472), 1.00 GiB | 25.83 MiB/s, done.
Resolving deltas: 100% (496283/496283), done.
fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [3]:
# Basic python and ML Libraries
import os
import random
import numpy as np
import pandas as pd
# for ignoring warnings
import warnings
warnings.filterwarnings('ignore')

# We will be reading images using OpenCV
import cv2

# xml library for parsing xml files
from xml.etree import ElementTree as et

# matplotlib for visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# torchvision libraries
import torch
from torch.utils.data import Subset
import torchvision
from torchvision import transforms as torchtrans  
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# these are the helper libraries imported.
from engine import train_one_epoch, evaluate
import utils
import transforms as T

# for image augmentations
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2


In [4]:
mkdir -p /kaggle/working/train-inriaperson && cp -r /kaggle/input/inriaperson/Train/Annotations/* /kaggle/working/train-inriaperson

In [5]:
cp -r /kaggle/input/inriaperson/Train/JPEGImages/* /kaggle/working/train-inriaperson

In [6]:
# defining the files directory and testing directory
train_dir = '/kaggle/working/train-inriaperson'
test_dir = '//kaggle/input/kitti-train-test-selection/Train_Test_Selection/Test'

In [7]:
#for xml-annotatation dataset
class ImagesDataset(torch.utils.data.Dataset):

    def __init__(self, files_dir, width, height, transforms=None):
        self.transforms = transforms
        self.files_dir = files_dir
        self.height = height
        self.width = width
            
        self.imgs = [image for image in sorted(os.listdir(files_dir)) if image.endswith(('.png', '.jpg'))]
        
        
        # classes: 0 index is reserved for background
        self.classes = ['background','Pedestrian']

    def __getitem__(self, idx):

        img_name = self.imgs[idx]
        image_path = os.path.join(self.files_dir, img_name)

        # reading the images and converting them to correct size and color    
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img_res = cv2.resize(img_rgb, (self.width, self.height), cv2.INTER_AREA)
        # diving by 255
        img_res /= 255.0
        
        # annotation file
        annot_filename = img_name[:-4] + '.xml'
        annot_file_path = os.path.join(self.files_dir, annot_filename)
        
        boxes = []
        labels = []
        tree = et.parse(annot_file_path)
        root = tree.getroot()
        
        # cv2 image gives size as height x width
        wt = img.shape[1]
        ht = img.shape[0]
        
        # box coordinates for xml files are extracted and corrected for image size given
        for member in root.findall('object'):
            label_text = member.find('name').text.lower()
            if label_text in ['ped', 'pedestrian','person']:
                labels.append(self.classes.index('Pedestrian')) 
            
            # bounding box
            xmin = int(member.find('bndbox').find('xmin').text)
            xmax = int(member.find('bndbox').find('xmax').text)
            
            ymin = int(member.find('bndbox').find('ymin').text)
            ymax = int(member.find('bndbox').find('ymax').text)
            
            
            xmin_corr = (xmin/wt)*self.width
            xmax_corr = (xmax/wt)*self.width
            ymin_corr = (ymin/ht)*self.height
            ymax_corr = (ymax/ht)*self.height
            
            boxes.append([xmin_corr, ymin_corr, xmax_corr, ymax_corr])
          
        
        # convert boxes into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        
        
        # getting the areas of the boxes
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        # suppose all instances are not crowd
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
        
        labels = torch.as_tensor(labels, dtype=torch.int64)


        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["area"] = area
        target["iscrowd"] = iscrowd
        # image_id
        image_id = idx
        target["image_id"] = image_id
        
  


        if self.transforms:
            
            sample = self.transforms(image = img_res,
                                     bboxes = target['boxes'],
                                     labels = labels)
            
            img_res = sample['image']
            target['boxes'] = torch.Tensor(sample['bboxes'])
            
            
            
        return img_res, target

    def __len__(self):
        return len(self.imgs)

In [8]:
#for txt-annotatation dataset
class KittiDataset(torch.utils.data.Dataset):
    def __init__(self, files_dir, width, height, transforms=None):
        self.transforms = transforms
        self.files_dir = files_dir
        self.height = height
        self.width = width
        self.imgs = [image for image in sorted(os.listdir(files_dir)) if image.endswith(('.png', '.jpg'))]
        self.classes = ['background', 'Pedestrian']

    def __getitem__(self, idx):
        img_name = self.imgs[idx]
        image_path = os.path.join(self.files_dir, img_name)

        # Reading the images and converting them to correct size and color
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img_res = cv2.resize(img_rgb, (self.width, self.height), cv2.INTER_AREA)
        img_res /= 255.0

        # Annotation file
        annot_filename = img_name[:-4] + '.txt'
        annot_file_path = os.path.join(self.files_dir, annot_filename)

        boxes = []
        labels = []

           # cv2 image gives size as height x width
        wt = img.shape[1]
        ht = img.shape[0]
        # Read annotation data from text file
        with open(annot_file_path, 'r') as file:
            for line in file:
                parts = line.strip().split(' ')
                label_text = parts[0].lower()

                if label_text in ['pedestrian', 'person']:
                    labels.append(self.classes.index('Pedestrian'))

                    # Bounding box coordinates
                    xmin = float(parts[4])
                    ymin = float(parts[5])
                    xmax = float(parts[6])
                    ymax = float(parts[7])

                    # Normalize coordinates
                    xmin_corr = (xmin/wt)*self.width
                    xmax_corr = (xmax/wt)*self.width
                    ymin_corr = (ymin/ht)*self.height
                    ymax_corr = (ymax/ht)*self.height

                    boxes.append([xmin_corr, ymin_corr, xmax_corr, ymax_corr])

        # Convert boxes into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        #print("boxes shape: ", boxes.size(), "at index",idx, img_name)
       
      
           

        # Calculate areas of the boxes
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        # Suppose all instances are not crowd
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)

        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["area"] = area
        target["iscrowd"] = iscrowd

        # image_id
        #image_id = torch.tensor([idx])
        image_id = idx
        target["image_id"] = image_id

        if self.transforms:
            sample = self.transforms(image=img_res, bboxes=target['boxes'], labels=labels)
            img_res = sample['image']
            target['boxes'] = torch.Tensor(sample['bboxes'])
        return img_res, target
    
    

    def __len__(self):
        return len(self.imgs)


In [9]:
# check train dataset
train_dataset = ImagesDataset(train_dir, 1920, 1080)
print('length of dataset = ', len(train_dataset), '\n')

# getting the image and target for a test index.  Feel free to change the index.
img, target = train_dataset[7]
print(img.shape, '\n',target)

length of dataset =  614 

(1080, 1920, 3) 
 {'boxes': tensor([[ 570.8836,  441.2571,  713.6045,  637.2000],
        [ 955.9608,  496.8000, 1120.2244,  683.4857]]), 'labels': tensor([1, 1]), 'area': tensor([27965.1406, 30665.6719]), 'iscrowd': tensor([0, 0]), 'image_id': 7}


In [10]:
# # check test dataset
test_dataset = KittiDataset(test_dir, 1920, 1080)
print('length of dataset = ', len(test_dataset), '\n')

# getting the image and target for a test index.  Feel free to change the index.
img, target = test_dataset[7]
print(img.shape, '\n',target)


length of dataset =  80 

(1080, 1920, 3) 
 {'boxes': tensor([[1128.2241,  475.8912, 1165.2483,  649.4400]]), 'labels': tensor([1]), 'area': tensor([6425.5000]), 'iscrowd': tensor([0]), 'image_id': 7}


In [11]:

def get_object_detection_model(num_classes):

    # load a model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 

    return model

In [12]:
# Send train=True fro training transforms and False for val/test transforms
def get_transform(train):
    
    if train:
        return A.Compose([
            A.HorizontalFlip(0.5),
            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5),
            A.Rotate(limit=15, p=0.5),
            ToTensorV2(p=1.0)
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))
    else:
        return A.Compose([
                            ToTensorV2(p=1.0)
                        ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [13]:
#Adjust the length of test dataset to align with other test datasets
# Define the total number of samples in your dataset
total_samples = len(train_dataset)

# Define the percentage of the dataset you want to load
portion_to_load = 0.5  # Example: load 50% of the dataset

# Calculate the number of samples to load
num_samples_to_load = int(total_samples * portion_to_load)

# Generate random indices to select a random subset of the dataset

indices = random.sample(range(total_samples), num_samples_to_load)

# Create a Subset dataset containing only the selected indices
subset = Subset(train_dataset, indices)
len(subset)


307

In [14]:
# use our dataset and defined transformations
train_dataset = ImagesDataset(train_dir, 1920, 1080, transforms= get_transform(train=True))
subset = Subset(train_dataset, indices)
train_dataset = subset
test_dataset = KittiDataset(test_dir, 1920, 1080, transforms= get_transform(train=False))

# define training and validation data loaders
data_loader_train = torch.utils.data.DataLoader(
    train_dataset, batch_size=4, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    test_dataset, batch_size=4, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

In [15]:
# to train on gpu if selected.
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


num_classes = 2

# get the model using our helper function
model = get_object_detection_model(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 133MB/s]


In [16]:
# training for 10 epochs
num_epochs = 10
for epoch in range(num_epochs):
    # training for one epoch
    train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Epoch: [0]  [ 0/77]  eta: 0:04:49  lr: 0.000071  loss: 1.3350 (1.3350)  loss_classifier: 0.9803 (0.9803)  loss_box_reg: 0.3134 (0.3134)  loss_objectness: 0.0375 (0.0375)  loss_rpn_box_reg: 0.0037 (0.0037)  time: 3.7534  data: 0.9091  max mem: 4799


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Epoch: [0]  [10/77]  eta: 0:01:01  lr: 0.000728  loss: 0.8653 (0.9255)  loss_classifier: 0.6454 (0.6298)  loss_box_reg: 0.2915 (0.2665)  loss_objectness: 0.0217 (0.0253)  loss_rpn_box_reg: 0.0037 (0.0039)  time: 0.9203  data: 0.1103  max mem: 4957


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


...

Epoch: [9]  [50/77]  eta: 0:00:18  lr: 0.000005  loss: 0.1142 (0.1178)  loss_classifier: 0.0457 (0.0440)  loss_box_reg: 0.0785 (0.0694)  loss_objectness: 0.0010 (0.0017)  loss_rpn_box_reg: 0.0026 (0.0027)  time: 0.6459  data: 0.0319  max mem: 4959


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Epoch: [9]  [60/77]  eta: 0:00:11  lr: 0.000005  loss: 0.1013 (0.1168)  loss_classifier: 0.0407 (0.0433)  loss_box_reg: 0.0604 (0.0693)  loss_objectness: 0.0005 (0.0016)  loss_rpn_box_reg: 0.0019 (0.0026)  time: 0.6454  data: 0.0322  max mem: 4959


libpng warning: iCCP: known incorrect sRGB profile


Epoch: [9]  [70/77]  eta: 0:00:04  lr: 0.000005  loss: 0.1132 (0.1177)  loss_classifier: 0.0415 (0.0438)  loss_box_reg: 0.0691 (0.0697)  loss_objectness: 0.0007 (0.0017)  loss_rpn_box_reg: 0.0022 (0.0026)  time: 0.6490  data: 0.0329  max mem: 4959
Epoch: [9]  [76/77]  eta: 0:00:00  lr: 0.000005  loss: 0.1202 (0.1194)  loss_classifier: 0.0420 (0.0445)  loss_box_reg: 0.0735 (0.0706)  loss_objectness: 0.0012 (0.0017)  loss_rpn_box_reg: 0.0021 (0.0026)  time: 0.6385  data: 0.0312  max mem: 4959
Epoch: [9] Total time: 0:00:51 (0.6653 s / it)
creating index...
index created!
Test:  [ 0/20]  eta: 0:00:20  model_time: 0.2752 (0.2752)  evaluator_time: 0.0031 (0.0031)  time: 1.0162  data: 0.6394  max mem: 4959
Test:  [19/20]  eta: 0:00:00  model_time: 0.2568 (0.2580)  evaluator_time: 0.0031 (0.0035)  time: 0.3512  data: 0.0579  max mem: 4959
Test: Total time: 0:00:07 (0.3530 s / it)
Averaged stats: model_time: 0.2568 (0.2580)  evaluator_time: 0.0031 (0.0035)
Accumulating evaluation results...
DO